In [1]:
import pandas as pd

monica = pd.read_excel('BANCOS/MONICA_PADRAO.xlsx')

risco_0 = baep[baep["FATOR_RISCO"] == 0]
risco_1 = baep[baep["FATOR_RISCO"] == 1]

with pd.ExcelWriter("pacientes_risco_sep.xlsx") as writer:
    risco_0.to_excel(writer, sheet_name="FATOR_RISCO_0", index=False)
    risco_1.to_excel(writer, sheet_name="FATOR_RISCO_1", index=False)


In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

dfs = pd.read_excel("pacientes_risco_sep.xlsx", sheet_name=None)

df = dfs["FATOR_RISCO_1"]

age = df["IDADE"]
MAP = df["PAM"]
sexo = df["SEXO"]
vop_real = df["VOP_MEDIDO"]

df["Eq.1"] = (
    6.52
    - 0.173 * age
    + 2.319e-3 * age**2
    - 1.355e-5 * age**2 * MAP
    + 1.430e-3 * age * MAP
    + 5.191e-3 * MAP
)

fator_sexo = np.where(sexo == 1, 0.0929, -0.0929)

df["Eq.2"] = (
    6.73
    - 0.171 * age
    + 2.267e-3 * age**2
    - 1.331e-5 * age**2 * MAP
    + 1.447e-3 * age * MAP
    + 2.360e-3 * MAP
    + fator_sexo
)

mse_eq1 = mean_squared_error(vop_real, df["Eq.1"])
rmse_eq1 = np.sqrt(mse_eq1)
mse_eq2 = mean_squared_error(vop_real, df["Eq.2"])
rmse_eq2 = np.sqrt(mse_eq2)

r2_eq1 = r2_score(vop_real, df["Eq.1"])
r2_eq2 = r2_score(vop_real, df["Eq.2"])


print("\n--- RMSE---")
print(f"RMSE Eq.1 = {rmse_eq1:.4f}")
print(f"RMSE Eq.2 = {rmse_eq2:.4f}")



--- APENAS RMSE (resumo) ---
RMSE Eq.1 = 1.1952
RMSE Eq.2 = 1.1924


In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

dfs = pd.read_excel("pacientes_risco_sep.xlsx", sheet_name=None)


df0 = dfs["FATOR_RISCO_0"]

age0 = df0["IDADE"]
MAP0 = df0["PAM"]
sexo0 = df0["SEXO"]
vop_real0 = df0["VOP"]


df0["Eq0.1"] = (
    4.43
    - 0.052 * age0
    + 0.00079 * age0**2
    + 0.00036 * age0 * MAP0
    + 0.0235 * MAP0
)

fator_sexo0 = np.where(sexo0 == 1, 0.188, -0.188)

df0["Eq0.2"] = (
    5.56
    - 0.065 * age0
    + 0.00071 * age0**2
    + 0.00058 * age0 * MAP0
    + 0.0101 * MAP0
    + fator_sexo0
)

df0["Erro_Quadrado_Eq0.1"] = (df0["Eq0.1"] - vop_real0)**2
df0["Erro_Quadrado_Eq0.2"] = (df0["Eq0.2"] - vop_real0)**2

dfs["FATOR_RISCO_0"] = df0


mse_eq0_1 = mean_squared_error(df0["VOP_MEDIDO"], df0["Eq0.1"])
rmse_eq0_1 = np.sqrt(mse_eq0_1)
r2_eq0_1 = r2_score(df0["VOP_MEDIDO"], df0["Eq0.1"])

mse_eq0_2 = mean_squared_error(df0["VOP_MEDIDO"], df0["Eq0.2"])
rmse_eq0_2 = np.sqrt(mse_eq0_2)
r2_eq0_2 = r2_score(df0["VOP_MEDIDO"], df0["Eq0.2"])

print("\n---  RMSE  ---")
print(f"RMSE Eq0.1 = {rmse_eq0_1:.4f}")
print(f"RMSE Eq0.2 = {rmse_eq0_2:.4f}")


KeyError: 'VOP'

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import pingouin as pg

baep = pd.read_excel('BANCOS/INDIOS_PADRAO.xlsx')

risco_0 = baep[baep["FATOR_RISCO"] == 0].copy()
risco_1 = baep[baep["FATOR_RISCO"] == 1].copy()

df = risco_1

age = df["IDADE"]
MAP = df["PAM"]
sexo = df["SEXO"]
vop_real = df["VOP"]

df["Eq.1"] = (
    6.52
    - 0.173 * age
    + 2.319e-3 * age**2
    - 1.355e-5 * age**2 * MAP
    + 1.430e-3 * age * MAP
    + 5.191e-3 * MAP
)

fator_sexo = np.where(sexo == 1, 0.0929, -0.0929)

df["Eq.2"] = (
    6.73
    - 0.171 * age
    + 2.267e-3 * age**2
    - 1.331e-5 * age**2 * MAP
    + 1.447e-3 * age * MAP
    + 2.360e-3 * MAP
    + fator_sexo
)

df["Erro_Quadrado_Eq1"] = (df["Eq.1"] - df["VOP"])**2
df["Erro_Quadrado_Eq2"] = (df["Eq.2"] - df["VOP"])**2

rmse_eq1 = np.sqrt(mean_squared_error(vop_real, df["Eq.1"]))
rmse_eq2 = np.sqrt(mean_squared_error(vop_real, df["Eq.2"]))
r2_eq1 = r2_score(vop_real, df["Eq.1"])
r2_eq2 = r2_score(vop_real, df["Eq.2"])

icc_df = pd.DataFrame({
    "id": np.arange(len(df)),
    "real": vop_real,
    "eq1": df["Eq.1"],
    "eq2": df["Eq.2"]
})

icc_eq1 = pg.intraclass_corr(
    data=icc_df.melt(id_vars="id", value_vars=["real", "eq1"]),
    targets="id", raters="variable", ratings="value"
)
icc_eq1 = icc_eq1[icc_eq1["Type"] == "ICC2"]["ICC"].values[0]

icc_eq2 = pg.intraclass_corr(
    data=icc_df.melt(id_vars="id", value_vars=["real", "eq2"]),
    targets="id", raters="variable", ratings="value"
)
icc_eq2 = icc_eq2[icc_eq2["Type"] == "ICC2"]["ICC"].values[0]

df0 = risco_0

age0 = df0["IDADE"]
MAP0 = df0["PAM"]
sexo0 = df0["SEXO"]
vop_real0 = df0["VOP"]

df0["Eq0.1"] = (
    4.43
    - 0.052 * age0
    + 0.00079 * age0**2
    + 0.00036 * age0 * MAP0
    + 0.0235 * MAP0
)

fator_sexo0 = np.where(sexo0 == 1, 0.188, -0.188)

df0["Eq0.2"] = (
    5.56
    - 0.065 * age0
    + 0.00071 * age0**2
    + 0.00058 * age0 * MAP0
    + 0.0101 * MAP0
    + fator_sexo0
)

df0["Erro_Quadrado_Eq0.1"] = (df0["Eq0.1"] - vop_real0)**2
df0["Erro_Quadrado_Eq0.2"] = (df0["Eq0.2"] - vop_real0)**2

rmse_eq0_1 = np.sqrt(mean_squared_error(vop_real0, df0["Eq0.1"]))
rmse_eq0_2 = np.sqrt(mean_squared_error(vop_real0, df0["Eq0.2"]))
r2_eq0_1 = r2_score(vop_real0, df0["Eq0.1"])
r2_eq0_2 = r2_score(vop_real0, df0["Eq0.2"])

icc0_df = pd.DataFrame({
    "id": np.arange(len(df0)),
    "real": vop_real0,
    "eq01": df0["Eq0.1"],
    "eq02": df0["Eq0.2"]
})

icc0_1 = pg.intraclass_corr(
    data=icc0_df.melt(id_vars="id", value_vars=["real", "eq01"]),
    targets="id", raters="variable", ratings="value"
)
icc0_1 = icc0_1[icc0_1["Type"] == "ICC2"]["ICC"].values[0]

icc0_2 = pg.intraclass_corr(
    data=icc0_df.melt(id_vars="id", value_vars=["real", "eq02"]),
    targets="id", raters="variable", ratings="value"
)
icc0_2 = icc0_2[icc0_2["Type"] == "ICC2"]["ICC"].values[0]

with pd.ExcelWriter("resultado_final_vop.xlsx") as writer:
    df0.to_excel(writer, sheet_name="FATOR_RISCO_0", index=False)
    df.to_excel(writer, sheet_name="FATOR_RISCO_1", index=False)

print("\n========== FATOR_RISCO_1 ==========")
print(f"Eq.1 → RMSE = {rmse_eq1:.4f} | R² = {r2_eq1:.4f} | ICC = {icc_eq1:.4f}")
print(f"Eq.2 → RMSE = {rmse_eq2:.4f} | R² = {r2_eq2:.4f} | ICC = {icc_eq2:.4f}")

print("\n========== FATOR_RISCO_0 ==========")
print(f"Eq0.1 → RMSE = {rmse_eq0_1:.4f} | R² = {r2_eq0_1:.4f} | ICC = {icc0_1:.4f}")
print(f"Eq0.2 → RMSE = {rmse_eq0_2:.4f} | R² = {r2_eq0_2:.4f} | ICC = {icc0_2:.4f}")



========== FATOR_RISCO_1 ==========
Eq.1 → RMSE = 2.3964 | R² = 0.0488 | ICC = 0.3658
Eq.2 → RMSE = 2.3822 | R² = 0.0600 | ICC = 0.3725

========== FATOR_RISCO_0 ==========
Eq0.1 → RMSE = 1.8868 | R² = 0.2766 | ICC = 0.5019
Eq0.2 → RMSE = 1.7884 | R² = 0.3501 | ICC = 0.5454


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

def icc_manual(df):
    Y = df.values
    n, k = Y.shape  # n indivíduos, k avaliadores (2)

    mean_target = np.mean(Y, axis=1, keepdims=True)
    mean_rater = np.mean(Y, axis=0, keepdims=True)
    grand_mean = np.mean(Y)

    SST = np.sum((Y - grand_mean) ** 2)
    SSB = np.sum((mean_target - grand_mean) ** 2) * k
    SSR = np.sum((mean_rater - grand_mean) ** 2) * n
    SSE = SST - SSB - SSR

    MSB = SSB / (n - 1)
    MSR = SSR / (k - 1)
    MSE = SSE / ((n - 1) * (k - 1))

    ICC = (MSB - MSE) / (MSB + (k - 1) * MSE + (k * (MSR - MSE) / n))
    return ICC

df = pd.read_excel("BANCOS/MONICA_PADRAO.xlsx")

df = df.rename(columns={
    "age": "IDADE",
    "MAP": "PAM"
})

df0 = df[df["FATOR_RISCO"] == 0].copy()   # sem fator
df1 = df[df["FATOR_RISCO"] == 1].copy()   # com fator

df0["Eq0"] = (
    4.6
    - 0.1 * df0["IDADE"]
    + 1.8e-03 * (df0["IDADE"] ** 2)
    + 6.0e-04 * (df0["IDADE"] * df0["PAM"])
    + 2.8e-02 * df0["PAM"]
)

df1["Eq1"] = (
    7.84
    - 0.33 * df1["IDADE"]
    + 3.8e-03 * (df1["IDADE"] ** 2)
    - 1.97e-05 * (df1["IDADE"] ** 2) * df1["PAM"]
    + 2.5e-03 * (df1["IDADE"] * df1["PAM"])
    - 1.9e-03 * df1["PAM"]
)


def calc_metrics(df, col_pred):
    rmse = np.sqrt(mean_squared_error(df["VOP"], df[col_pred]))
    r2 = r2_score(df["VOP"], df[col_pred])
    icc = icc_manual(df[["VOP", col_pred]])
    return rmse, r2, icc


rmse0, r20, icc0 = calc_metrics(df0, "Eq0")
rmse1, r21, icc1 = calc_metrics(df1, "Eq1")


with pd.ExcelWriter("VOP_RESULTADOS.xlsx") as writer:
    df0.to_excel(writer, sheet_name="SEM_FATOR", index=False)
    df1.to_excel(writer, sheet_name="COM_FATOR", index=False)



print("\n========== SEM FATOR (FATOR_RISCO = 0) ==========")
print(f"Eq0 → RMSE = {rmse0:.4f} | R² = {r20:.4f} | ICC = {icc0:.4f}")

print("\n========== COM FATOR (FATOR_RISCO = 1) ==========")
print(f"Eq1 → RMSE = {rmse1:.4f} | R² = {r21:.4f} | ICC = {icc1:.4f}")



========== SEM FATOR (FATOR_RISCO = 0) ==========
Eq0 → RMSE = 1.6851 | R² = -0.2103 | ICC = 0.3233

========== COM FATOR (FATOR_RISCO = 1) ==========
Eq1 → RMSE = 2.6168 | R² = -0.4092 | ICC = 0.3760
